In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import json
import time
import gmaps
from scipy.stats import linregress
from pprint import pprint

In [2]:
# Reading the data 
covid_clean_CA_TX = pd.read_csv("../Resources/covid_clean_CA_TX.csv")
covid_clean_CA_TX

,State,countyFIPS,County Name,population,stateFIPS,1/22/2020,1/23/2020,1/24/2020,1/25/2020,1/26/2020,...,10/21/2020,10/22/2020,10/23/2020,10/24/2020,10/25/2020,10/26/2020,10/27/2020,10/28/2020,10/29/2020,10/30/2020
0,CA,6000,Grand Princess Cruise Ship,0,6,0,0,0,0,0,...,21,21,21,21,21,21,21,21,21,21
1,CA,6001,Alameda County,1671329,6,0,0,0,0,0,...,22932,23001,23133,23215,23312,23391,23391,23576,23636,23775
2,CA,6003,Alpine County,1129,6,0,0,0,0,0,...,3,3,3,3,3,3,3,3,3,3
3,CA,6005,Amador County,39752,6,0,0,0,0,0,...,276,287,291,291,291,299,301,302,308,311
4,CA,6007,Butte County,219186,6,0,0,0,0,0,...,3018,3030,3047,3047,3047,3073,3073,3113,3127,3144
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
308,TX,48499,Wood County,45539,48,0,0,0,0,0,...,621,623,626,626,626,633,636,639,655,655
309,TX,48501,Yoakum County,8713,48,0,0,0,0,0,...,289,289,289,289,289,289,289,300,304,319
310,TX,48503,Young County,18010,48,0,0,0,0,0,...,497,507,521,535,535,535,559,578,591,605
311,TX,48505,Zapata County,14179,48,0,0,0,0,0,...,342,342,343,345,346,346,352,352,352,357


In [3]:
#Extracting head information
CA_TX_population = covid_clean_CA_TX[covid_clean_CA_TX.columns[0:4]]
CA_TX_population

,State,countyFIPS,County Name,population
0,CA,6000,Grand Princess Cruise Ship,0
1,CA,6001,Alameda County,1671329
2,CA,6003,Alpine County,1129
3,CA,6005,Amador County,39752
4,CA,6007,Butte County,219186
...,...,...,...,...
308,TX,48499,Wood County,45539
309,TX,48501,Yoakum County,8713
310,TX,48503,Young County,18010
311,TX,48505,Zapata County,14179


In [4]:
#Dropping columns for future biweekly count
daily_cases= covid_clean_CA_TX.drop(columns=["State", 'countyFIPS','County Name','stateFIPS','population'])
daily_cases

,1/22/2020,1/23/2020,1/24/2020,1/25/2020,1/26/2020,1/27/2020,1/28/2020,1/29/2020,1/30/2020,1/31/2020,...,10/21/2020,10/22/2020,10/23/2020,10/24/2020,10/25/2020,10/26/2020,10/27/2020,10/28/2020,10/29/2020,10/30/2020
0,0,0,0,0,0,0,0,0,0,0,...,21,21,21,21,21,21,21,21,21,21
1,0,0,0,0,0,0,0,0,0,0,...,22932,23001,23133,23215,23312,23391,23391,23576,23636,23775
2,0,0,0,0,0,0,0,0,0,0,...,3,3,3,3,3,3,3,3,3,3
3,0,0,0,0,0,0,0,0,0,0,...,276,287,291,291,291,299,301,302,308,311
4,0,0,0,0,0,0,0,0,0,0,...,3018,3030,3047,3047,3047,3073,3073,3113,3127,3144
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
308,0,0,0,0,0,0,0,0,0,0,...,621,623,626,626,626,633,636,639,655,655
309,0,0,0,0,0,0,0,0,0,0,...,289,289,289,289,289,289,289,300,304,319
310,0,0,0,0,0,0,0,0,0,0,...,497,507,521,535,535,535,559,578,591,605
311,0,0,0,0,0,0,0,0,0,0,...,342,342,343,345,346,346,352,352,352,357


In [5]:
#Extracting cases by the end of every two weeks for biweekly count
biweekly_cases_count = daily_cases[daily_cases.columns[::14]]
biweekly_cases_count

,1/22/2020,2/5/2020,2/19/2020,3/4/2020,3/18/2020,4/1/2020,4/15/2020,4/29/2020,5/13/2020,5/27/2020,...,6/24/2020,7/8/2020,7/22/2020,8/5/2020,8/19/2020,9/2/2020,9/16/2020,9/30/2020,10/14/2020,10/28/2020
0,0,0,0,0,21,21,21,21,21,21,...,21,21,21,21,21,21,21,21,21,21
1,0,0,0,1,31,359,962,1568,2178,3049,...,5275,7245,9643,12136,15836,18695,20097,21383,22325,23576
2,0,0,0,0,0,1,2,2,2,2,...,2,2,2,2,2,2,2,2,3,3
3,0,0,0,0,1,3,7,8,9,10,...,19,35,65,156,218,237,247,257,271,302
4,0,0,0,0,0,9,16,16,20,40,...,127,295,683,1029,1494,2208,2638,2804,2948,3113
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
308,0,0,0,0,0,1,6,7,15,25,...,77,115,196,313,373,408,472,526,579,639
309,0,0,0,0,0,0,0,1,2,3,...,15,40,55,90,129,155,198,224,278,300
310,0,0,0,0,0,3,4,4,4,4,...,22,38,81,151,203,226,258,332,404,578
311,0,0,0,0,0,0,4,7,7,8,...,40,63,115,165,231,281,321,333,342,352


In [6]:
#Biweekly count 
biweekly_cases_count_for_merge = biweekly_cases_count.diff(axis=1, periods=1)
biweekly_cases_count_for_merge 

,1/22/2020,2/5/2020,2/19/2020,3/4/2020,3/18/2020,4/1/2020,4/15/2020,4/29/2020,5/13/2020,5/27/2020,...,6/24/2020,7/8/2020,7/22/2020,8/5/2020,8/19/2020,9/2/2020,9/16/2020,9/30/2020,10/14/2020,10/28/2020
0,NaN,0.0,0.0,0.0,21.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,NaN,0.0,0.0,1.0,30.0,328.0,603.0,606.0,610.0,871.0,...,1242.0,1970.0,2398.0,2493.0,3700.0,2859.0,1402.0,1286.0,942.0,1251.0
2,NaN,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,NaN,0.0,0.0,0.0,1.0,2.0,4.0,1.0,1.0,1.0,...,9.0,16.0,30.0,91.0,62.0,19.0,10.0,10.0,14.0,31.0
4,NaN,0.0,0.0,0.0,0.0,9.0,7.0,0.0,4.0,20.0,...,54.0,168.0,388.0,346.0,465.0,714.0,430.0,166.0,144.0,165.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
308,NaN,0.0,0.0,0.0,0.0,1.0,5.0,1.0,8.0,10.0,...,26.0,38.0,81.0,117.0,60.0,35.0,64.0,54.0,53.0,60.0
309,NaN,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,...,7.0,25.0,15.0,35.0,39.0,26.0,43.0,26.0,54.0,22.0
310,NaN,0.0,0.0,0.0,0.0,3.0,1.0,0.0,0.0,0.0,...,11.0,16.0,43.0,70.0,52.0,23.0,32.0,74.0,72.0,174.0
311,NaN,0.0,0.0,0.0,0.0,0.0,4.0,3.0,0.0,1.0,...,28.0,23.0,52.0,50.0,66.0,50.0,40.0,12.0,9.0,10.0


In [7]:
#dropping first date in the dataframe for future merging two dataframes
biweekly_cases_count_for_merge = biweekly_cases_count_for_merge.drop(columns=["1/22/2020"])
biweekly_cases_count_for_merge 

,2/5/2020,2/19/2020,3/4/2020,3/18/2020,4/1/2020,4/15/2020,4/29/2020,5/13/2020,5/27/2020,6/10/2020,6/24/2020,7/8/2020,7/22/2020,8/5/2020,8/19/2020,9/2/2020,9/16/2020,9/30/2020,10/14/2020,10/28/2020
0,0.0,0.0,0.0,21.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,1.0,30.0,328.0,603.0,606.0,610.0,871.0,984.0,1242.0,1970.0,2398.0,2493.0,3700.0,2859.0,1402.0,1286.0,942.0,1251.0
2,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,0.0,1.0,2.0,4.0,1.0,1.0,1.0,0.0,9.0,16.0,30.0,91.0,62.0,19.0,10.0,10.0,14.0,31.0
4,0.0,0.0,0.0,0.0,9.0,7.0,0.0,4.0,20.0,33.0,54.0,168.0,388.0,346.0,465.0,714.0,430.0,166.0,144.0,165.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
308,0.0,0.0,0.0,0.0,1.0,5.0,1.0,8.0,10.0,26.0,26.0,38.0,81.0,117.0,60.0,35.0,64.0,54.0,53.0,60.0
309,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,5.0,7.0,25.0,15.0,35.0,39.0,26.0,43.0,26.0,54.0,22.0
310,0.0,0.0,0.0,0.0,3.0,1.0,0.0,0.0,0.0,7.0,11.0,16.0,43.0,70.0,52.0,23.0,32.0,74.0,72.0,174.0
311,0.0,0.0,0.0,0.0,0.0,4.0,3.0,0.0,1.0,4.0,28.0,23.0,52.0,50.0,66.0,50.0,40.0,12.0,9.0,10.0


In [8]:
#Merging dataframes
biweekly_cases_CA_TX = CA_TX_population.join(biweekly_cases_count_for_merge, how="outer")
biweekly_cases_CA_TX

,State,countyFIPS,County Name,population,2/5/2020,2/19/2020,3/4/2020,3/18/2020,4/1/2020,4/15/2020,...,6/24/2020,7/8/2020,7/22/2020,8/5/2020,8/19/2020,9/2/2020,9/16/2020,9/30/2020,10/14/2020,10/28/2020
0,CA,6000,Grand Princess Cruise Ship,0,0.0,0.0,0.0,21.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,CA,6001,Alameda County,1671329,0.0,0.0,1.0,30.0,328.0,603.0,...,1242.0,1970.0,2398.0,2493.0,3700.0,2859.0,1402.0,1286.0,942.0,1251.0
2,CA,6003,Alpine County,1129,0.0,0.0,0.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,CA,6005,Amador County,39752,0.0,0.0,0.0,1.0,2.0,4.0,...,9.0,16.0,30.0,91.0,62.0,19.0,10.0,10.0,14.0,31.0
4,CA,6007,Butte County,219186,0.0,0.0,0.0,0.0,9.0,7.0,...,54.0,168.0,388.0,346.0,465.0,714.0,430.0,166.0,144.0,165.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
308,TX,48499,Wood County,45539,0.0,0.0,0.0,0.0,1.0,5.0,...,26.0,38.0,81.0,117.0,60.0,35.0,64.0,54.0,53.0,60.0
309,TX,48501,Yoakum County,8713,0.0,0.0,0.0,0.0,0.0,0.0,...,7.0,25.0,15.0,35.0,39.0,26.0,43.0,26.0,54.0,22.0
310,TX,48503,Young County,18010,0.0,0.0,0.0,0.0,3.0,1.0,...,11.0,16.0,43.0,70.0,52.0,23.0,32.0,74.0,72.0,174.0
311,TX,48505,Zapata County,14179,0.0,0.0,0.0,0.0,0.0,4.0,...,28.0,23.0,52.0,50.0,66.0,50.0,40.0,12.0,9.0,10.0


In [9]:
#Saving the final data
biweekly_cases_CA_TX.to_csv("../Resources/biweekly_cases_CA_TX.csv", index=False, header=True)